In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import pandas as pd
import re
import moviepy

In [11]:
u = conn.as_pandas('select * from u52')

In [13]:
u.dtypes

wins                int64
local              object
opponent           object
local_points        int64
opponent_points     int64
duration           object
date               object
event              object
category           object
round              object
has_video           int64
url_video          object
dtype: object

In [5]:
df.head()

,wins,local,opponent,local_points,opponent_points,duration,date,event,category,round,has_video,url_video
0,1,BUCHARD Amandine,VAN SNICK Charline,10,0,00:00,20 Sep 2018,World Championships Baku 2018,-52 kg,Bronze,True,https://judobase.ijf.org/#/competition/contest...
1,0,BUCHARD Amandine,ABE Uta,0,100,03:34,20 Sep 2018,World Championships Baku 2018,-52 kg,Semi-Final,True,https://judobase.ijf.org/#/competition/contest...
2,1,BUCHARD Amandine,PRIMO Gefen,10,0,00:00,20 Sep 2018,World Championships Baku 2018,-52 kg,Quarter-Final,True,https://judobase.ijf.org/#/competition/contest...
3,1,BUCHARD Amandine,RAMOS Joana,110,0,00:14,20 Sep 2018,World Championships Baku 2018,-52 kg,Round 3,True,https://judobase.ijf.org/#/competition/contest...
4,1,BUCHARD Amandine,WU Shugen,110,0,02:04,20 Sep 2018,World Championships Baku 2018,-52 kg,Round 2,True,https://judobase.ijf.org/#/competition/contest...


In [46]:
df = pd.read_csv('data66kg.csv', index_col=0)

In [51]:
import datetime

In [57]:
datetime.datetime.strptime('Jun 1 2015  1:33PM', '%b %d %Y %I:%M%p')

datetime.datetime(2015, 6, 1, 13, 33)

In [65]:
datetime.datetime.strptime('26 Apr 2015', '%d %b %Y') - datetime.datetime.strptime('26 Jun 2015', '%d %b %Y')

datetime.timedelta(-61)

In [5]:
# abrir Google Chrome
driver = webdriver.Chrome(r"C:\Users\manuel.lopez\Documents\chromedriver.exe")
try:
    driver.maximize_window()
except:
    pass

In [26]:
df[df.event.str.contains('Europe')]

,wins,local,opponent,local_points,opponent_points,duration,date,event,category,round,has_video,url_video,competition_type
11,0,MARGVELASHVILI Vazha,ADAM Michel,0,10,05:00,26 Apr 2018,European Championships Tel Aviv 2018,-66 kg,Round 1,True,https://judobase.ijf.org/#/competition/contest...,None
38,0,MARGVELASHVILI Vazha,POLIAK Matej,0,100,02:20,20 Apr 2017,European Championships Warsaw 2017,-66 kg,Round 2,False,NaN,None
39,1,MARGVELASHVILI Vazha,URIARTE Gar,120,0,00:53,20 Apr 2017,European Championships Warsaw 2017,-66 kg,Round 1,False,NaN,None
12,1,FLICKER Tal,GUSIC Marko,100,0,00:00,26 Apr 2018,European Championships Tel Aviv 2018,-66 kg,Bronze,True,https://judobase.ijf.org/#/competition/contest...,None
13,1,FLICKER Tal,CRISOSTOMO Joao,100,10,02:58,26 Apr 2018,European Championships Tel Aviv 2018,-66 kg,Repechage,True,https://judobase.ijf.org/#/competition/contest...,None
14,0,FLICKER Tal,GOMBOC Adrian,0,10,05:00,26 Apr 2018,European Championships Tel Aviv 2018,-66 kg,Quarter-Final,True,https://judobase.ijf.org/#/competition/contest...,None
15,1,FLICKER Tal,BUNESCU Vadim,100,0,02:25,26 Apr 2018,European Championships Tel Aviv 2018,-66 kg,Round 2,True,https://judobase.ijf.org/#/competition/contest...,None
16,1,FLICKER Tal,BURNS Nathon,10,0,05:00,26 Apr 2018,European Championships Tel Aviv 2018,-66 kg,Round 1,True,https://judobase.ijf.org/#/competition/contest...,None
37,0,FLICKER Tal,MINKOU Dzmitry,0,100,00:00,20 Apr 2017,European Championships Warsaw 2017,-66 kg,Round 2,False,NaN,None
34,1,AN Baul,MEDVES Matteo,110,0,03:45,18 Feb 2017,European Open Rome 2017,-66 kg,Final,False,NaN,None


In [18]:

df['competition_type'] = df.event.apply(lambda event: next((c for c in competition_types if c in event), 'None'))


In [28]:
df[df.competition_type=='None'].event.apply(lambda e: ' '.join(e.split()[2:3])).value_counts()

Minsk               91
Tel                 81
Prague              79
Tunis               76
Oberwart            69
Warsaw              65
Sofia               60
San                 55
Sarajevo            53
Hong                50
Belgrade            49
Buenos              48
Rome                48
Celje-Podcetrtek    46
Jakarta             41
Lima                40
Bucharest           40
Berlin              39
Madrid              38
Santiago            35
Casablanca          33
Katowice            30
Taipei              30
Panama              28
Odivelas            28
Saarbrucken         27
Leibnitz            26
Kazan               25
Glasgow             23
Antananarivo        22
                    ..
Uster                7
Cluj-Napoca          6
Uster-Zurich         6
Wroclaw              6
Perth                5
La                   5
Montevideo           5
Maribor              5
Deva                 5
Almaty               4
Nuku?alofa           4
Isskul               4
Kuwait     

In [20]:
df.competition_type.value_counts()

None                   1920
Grand Prix             1189
Grand Slam              652
World Championships     372
Masters                  58
Olympic Games            45
Name: competition_type, dtype: int64

In [19]:
df.head()

,wins,local,opponent,local_points,opponent_points,duration,date,event,category,round,has_video,url_video,competition_type
0,1,ABE Hifumi,SERIKZHANOV Yerlan,100,0,02:47,20 Sep 2018,World Championships Baku 2018,-66 kg,Final,True,https://judobase.ijf.org/#/competition/contest...,World Championships
1,1,ABE Hifumi,AN Baul,10,0,02:38,20 Sep 2018,World Championships Baku 2018,-66 kg,Semi-Final,True,https://judobase.ijf.org/#/competition/contest...,World Championships
2,1,ABE Hifumi,ZANTARAIA Georgii,100,0,03:10,20 Sep 2018,World Championships Baku 2018,-66 kg,Quarter-Final,True,https://judobase.ijf.org/#/competition/contest...,World Championships
3,1,ABE Hifumi,VIERU Denis,10,0,00:00,20 Sep 2018,World Championships Baku 2018,-66 kg,Round 4,True,https://judobase.ijf.org/#/competition/contest...,World Championships
4,1,ABE Hifumi,MEDVES Matteo,100,0,02:57,20 Sep 2018,World Championships Baku 2018,-66 kg,Round 3,True,https://judobase.ijf.org/#/competition/contest...,World Championships


### Dada url video judobase, extraer info y acceder al youtube

In [14]:
# elige una random y ábrela
while True:
    url = df.sample(1).iloc[0].url_video
    if not pd.isnull(url):
        break

In [15]:
driver.get(url)
time.sleep(.5)

### Extraer info

In [16]:
# extract names
local, opponent = map(lambda x: x.text, driver.find_elements_by_class_name('col-xs-6'))
local, opponent = map(lambda x: x[:x.find('\n')], [local, opponent])

In [17]:
local

'MATEO Wander'

In [18]:
opponent

'ALTYNBEK UULU Almaz'

In [19]:
# tabla con puntuaciones etiquetadas en tiempo
tabla_timed_points = driver.find_elements_by_tag_name('tbody')[1]

In [20]:
filas = tabla_timed_points.find_elements_by_tag_name('tr')

In [21]:
nrows = len(filas[0].find_elements_by_tag_name('td'))
nrows

3

In [22]:
if nrows == 2:
    for row_number, row in enumerate(filas):
        for col_number, value in enumerate(row.find_elements_by_tag_name('td')):
            print(row_number, col_number, value.text)

else:
    print(f'{nrows} rows!')

3 rows!


In [23]:
youtube_frame = driver.find_element_by_class_name('js-media')

In [24]:
driver.switch_to.frame(youtube_frame)

In [25]:
url_youtube = driver.find_element_by_class_name('ytp-title-link').get_attribute('href')

url_youtube

'https://www.youtube.com/watch?v=TgFQMxD4aNs'

In [26]:
seconds = 22

In [27]:
url_youtube_time = url_youtube + '&t=' + str(seconds)
url_youtube_time

'https://www.youtube.com/watch?v=TgFQMxD4aNs&t=22'

In [28]:
driver.get(url_youtube_time)

### Youtube downloads

In [29]:
from pytube import YouTube

In [30]:
y = YouTube(url_youtube_time)

In [31]:
[d for d in dir(y) if d[0]!='_']

['age_restricted',
 'caption_tracks',
 'captions',
 'description',
 'embed_html',
 'embed_url',
 'fmt_streams',
 'init',
 'initialize_caption_objects',
 'initialize_stream_objects',
 'js',
 'js_url',
 'length',
 'player_config_args',
 'prefetch',
 'prefetch_init',
 'rating',
 'register_on_complete_callback',
 'register_on_progress_callback',
 'stream_monostate',
 'streams',
 'thumbnail_url',
 'title',
 'vid_descr',
 'vid_info',
 'vid_info_url',
 'video_id',
 'views',
 'watch_html',
 'watch_url']

In [32]:
video = y.streams.filter(res='720p', mime_type='video/mp4').first()

In [33]:
assert video is not None, 'No video'

In [34]:
video.download(filename='judo720p3')

'C:\\Users\\manuel.lopez\\Documents\\scrape\\0e6a72df62d07aa6cc043c46d63ad5e7\\judo720p3.mp4'

### Edición con moviepy

In [1]:
3

3

In [3]:
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip, concatenate_videoclips

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
# Load myHolidays.mp4 and select the subclip 00:00:50 - 00:00:60
clip = VideoFileClip('./my_concatenation.mp4')

In [9]:
clip.duration

15.05

In [14]:
corte = clip.subclip(4,8)

In [12]:
text0 = TextClip("Primer chunk", fontsize=70, color='red').set_position(('right', 'top'), relative=True).set_duration(2)
text1 = TextClip("Segundo chunk", fontsize=70, color='blue').set_position((0.4, 0), relative=True).set_duration(2)
# text2 = TextClip("Tercer chunk", fontsize=70, color='green').set_position((0.8, 0), relative=True).set_duration(5)

# textos = (text0, text1, text2)

In [13]:
textos = concatenate_videoclips([text0, text1])

In [15]:
final_clip = CompositeVideoClip([corte, textos])

In [16]:
final_clip.duration

4

In [17]:
final_clip.write_videofile("my_concatenatio2n2.mp4")

[MoviePy] >>>> Building video my_concatenatio2n2.mp4
[MoviePy] Writing audio in my_concatenatio2n2TEMP_MPY_wvf_snd.mp3


100%|█████████████████████████████████████████████████████████████████████████████████| 89/89 [00:00<00:00, 271.76it/s]


[MoviePy] Done.
[MoviePy] Writing video my_concatenatio2n2.mp4


 99%|███████████████████████████████████████████████████████████████████████████████▏| 100/101 [00:06<00:00, 12.44it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: my_concatenatio2n2.mp4 

